In [1]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# importing modules

from pywebio.input import *
from pywebio.output import *
from pywebio import start_server
from pywebio.session import set_env, download
from docx import Document
import os
import openai

openai.api_key = "OPEN_API_KEY"

def TOC(topic, toc_seed, style):   

    # Build a table of contents
    
    intro = topic + "\n1. " + toc_seed + "\n2."
    toc_response = openai.Completion.create(
        engine="davinci",
        prompt=intro,
        temperature=0.5,
        max_tokens=70,
        top_p=1,
        frequency_penalty=0.52,
        presence_penalty=0.5,
        stop=["11."]
    )
    
    TOC=[]
    TOC="Title: "+ intro + (toc_response["choices"][0]["text"])
    TOC=(TOC.split("\n"))
    TOC=[x for x in TOC if x]

    # Write an essay

    followUp=[]
    followUp=[e[3:] for e in TOC[1:]]
    followUp=["Write a " + style + " on the topic of " +topic +" and "+ str(x) + "\n\"\"\"\"\"\"\n" + str(x)+"\n\"\"\"\"\"\"\nThis is the " + style + " I wrote about " +str(x) + "\n\"\"\"\"\"\"" for x in followUp]
       
    content_response = openai.Completion.create(
      engine="davinci-instruct-beta",
      prompt=followUp,
      temperature=0.5,
      max_tokens=300,
      top_p=1.0,
      frequency_penalty=0.52,
      presence_penalty=0.5,
    )
    
    #output markdown file
    
    f = open(topic + ".md", "a")

    f.write("# " + topic)
    f.write("\n> This was made using [open.ai](open.ai)")

    for i in range(0,len(content_response["choices"])):
        f.write("\n \n")
        f.write("## " + TOC[i+1])
        f.write(content_response["choices"][i]["text"])
        f.write("\n \n")
     
    f.close()
    
    
    
    #output word document

    document = Document()

    document.add_heading(TOC[0],0)

    for i in range(0,len(content_response["choices"])):
        document.add_paragraph()
        document.add_heading(TOC[i+1],1)
        document.add_paragraph(content_response["choices"][i]["text"])

        document.save(topic + '.docx')
    
    
    put_markdown("\n This text was made using [open.ai](open.ai)")
    put_markdown("\n You have generated some text based on " + topic + ". Copy and paste this markdown to save it. It is very likely that you will need to edit it!")
    put_link("\nClick to try again", url="/")
    put_markdown("\n\n\n\n# " + topic)

    for i in range(0,len(content_response["choices"])):
        put_markdown("\n \n")
        put_markdown("## " + TOC[i+1])
        put_markdown(content_response["choices"][i]["text"])
        put_markdown("\n \n")
    

def main():
    
    set_env(output_animation=False)
    
    put_markdown('# OpenAI Writing Partner')
    
    
    data = input_group("Let´s Write Something!",[
    input("What do you want to write about?", name='topic', value="Getting Started With Github"),
    input("Tell me the first entry of your table of contents", name='toc_seed', value="Introduction" ),
    select("Select writing style", ['Professional Report', 'University paper','High-school essay','Manual' ], name='style')
    ])    

    
    TOC(data['topic'], data['toc_seed'], data['style'])  
    
    
start_server(main, port=8080, debug=True, cdn=False)
session.hold()

Listen on 0.0.0.0:8080
